In [235]:
from collections import deque
from queue import PriorityQueue

In [236]:
# Reference used a building ground and inspiration
# https://github.com/speix/8-puzzle-solver

In [237]:
# Representing a 'State'
class State:
    def __init__(self, state, parent, move, moved, depth, cost):
        self.state = state
        self.parent = parent
        self.move = move
        self.moved = moved
        self.depth = depth
        self.cost = cost
        if self.state:
            self.map = ''.join(str(e) for e in self.state) # say, [1, 4, 2, 5, 3, 0] becomes '142530'
        
    def __eq__(self, other): # override equality check for comparing two state
        return self.map == other.map
        
    def __lt__(self, other): # override 'less than' operator
        return self.map < other.map
        
    def __str__(self):
        pieces = self.state
        out = ''
        for i in range(0, len(pieces)):
            if i == 0:
                out += '-------------\n'
            out = out + '| {0:d} '.format(pieces[i])
            if (i+1) % board_len == 0:
                out = out + '|' + '\n' + '-------------\n'
        return out


In [238]:
# Representing the state configurations of our 3x2 six-puzzle

# Assume that element "0" is an unoccupied tile

initial_state = [1, 4, 2, 5, 3, 0]
goal_state = [0, 1, 2, 5, 4, 3]

goal_node = None

board_len = 3

moves = {
    1: "Up",
    2: "Down",
    3: "Left",
    4: "Right"
}

def showPuzzle(pieces):
    out = ''
    for i in range(0, len(pieces)):
        if i == 0:
            out += '-------------\n'
        #out = out + '| %d ' % (pieces[i])
        out = out + '| {0:d} '.format(pieces[i])
        if (i+1) % board_len == 0:
            out = out + '|' + '\n' + '-------------\n'
    print(out)

In [298]:
# Generate children
def expandChildren(node):
    
    neighbors = []
    
    moved_piece, new_state = move(node.state, 1)
    
    print(moved_piece)
    
    neighbors.append(State(new_state, node, 1, moved_piece, node.depth + 1, node.cost + 1))
    
    moved_piece, new_state = move(node.state, 2)
    neighbors.append(State(new_state, node, 2, moved_piece, node.depth + 1, node.cost + 1))
    
    moved_piece, new_state = move(node.state, 3)
    neighbors.append(State(new_state, node, 3, moved_piece, node.depth + 1, node.cost + 1))
    
    moved_piece, new_state = move(node.state, 4)
    neighbors.append(State(new_state, node, 4, moved_piece, node.depth + 1, node.cost + 1))
    
    nodes = [neighbor for neighbor in neighbors if neighbor.state]
    
    return nodes

# Move a piece
def move(state, direction):

    new_state = state[:] # makes a copy of current board configuration

    index = new_state.index(0)
    
    if index == 0: # Moving top-left piece
        if direction == 1:
            return 100, None
        if direction == 2:
            temp = new_state[0]
            new_state[0] = new_state[3] # 3 as in the index of the piece right below the top leftmost piece
            new_state[3] = temp
            return new_state[3], new_state
        if direction == 3:
            return 100, None
        if direction == 4:
            temp = new_state[0]
            new_state[0] = new_state[1]
            new_state[1] = temp
            return new_state[1], new_state
        
    if index == 1: # Moving top-middle piece
        if direction == 1:
            return 100, None
        if direction == 2:
            temp = new_state[1]
            new_state[1] = new_state[4] 
            new_state[4] = temp
            return new_state[4], new_state
        if direction == 3:
            temp = new_state[1]
            new_state[1] = new_state[0]
            new_state[0] = temp
            return new_state[0], new_state
        if direction == 4:
            temp = new_state[1]
            new_state[1] = new_state[2]
            new_state[2] = temp
            return new_state[2], new_state
        
    if index == 2: # Moving top-right piece
        if direction == 1:
            return 100, None
        if direction == 2:
            temp = new_state[2]
            new_state[2] = new_state[5] 
            new_state[5] = temp
            return new_state[5], new_state
        if direction == 3:
            temp = new_state[2]
            new_state[2] = new_state[1]
            new_state[1] = temp
            return new_state[1], new_state
        if direction == 4:
            return 100, None
        
    if index == 3: # Moving bottom-left piece
        if direction == 1:
            temp = new_state[3]
            new_state[3] = new_state[0] 
            new_state[0] = temp
            return new_state[0], new_state
        if direction == 2:
            return 100, None
        if direction == 3:
            return 100, None
        if direction == 4:
            temp = new_state[3]
            new_state[3] = new_state[4]
            new_state[4] = temp
            return new_state[4], new_state
    
    if index == 4: # Moving bottom-middle piece
        if direction == 1:
            temp = new_state[4]
            new_state[4] = new_state[1]
            new_state[1] = temp
            return new_state[1], new_state
        if direction == 2:
            return 100, None
        if direction == 3:
            temp = new_state[4]
            new_state[4] = new_state[3]
            new_state[3] = temp
            return new_state[3], new_state
        if direction == 4:
            temp = new_state[4]
            new_state[4] = new_state[5]
            new_state[5] = temp
            return new_state[5], new_state
        
    if index == 5: # Moving bottom-right piece
        if direction == 1:
            temp = new_state[5]
            new_state[5] = new_state[2]
            new_state[2] = temp
            return new_state[2], new_state
        if direction == 2:
            return 100, None
        if direction == 3:
            temp = new_state[5]
            new_state[5] = new_state[4]
            new_state[4] = temp
            return new_state[4], new_state
        if direction == 4:
            return 100, None

    

In [299]:
# BFS

def bfs(init_state):
    
    global goal_node
    
    explored = set()
    
    start_state = State(init_state, None, None, 0, 0, 0)
    q = deque([start_state])
        
    print("Start state:")
    print(start_state)
    
    while q:
        node = q.popleft()
        
        explored.add(node.map)
        
        if node.state == goal_state:
            goal_node = node
            return q
    
        neighbors = expandChildren(node)
        neighbors.sort(key=lambda s: s.moved)
        
        for n in neighbors: 
            if n.map not in explored:
                q.append(n)
                explored.add(n.map)


In [300]:
# Uniform Cost Search

def ucs(init_state):
    
    global goal_node
    
    explored = set()
    
    q = PriorityQueue()
    start_state = State(init_state, None, None, 0, 0, 0)
    q.put((0, start_state))
        
    print("Start state:")
    print(start_state)
    
    while q:
        
        pq_priority, node = q.get()
        
        explored.add(node.map)
        
        if node.state == goal_state:
            goal_node = node
            return q
    
        neighbors = expandChildren(node)
        
        for n in neighbors: 
            if n.map not in explored:
                q.put((n.cost, n))
                explored.add(n.map)

In [301]:
# DFS

def dfs(init_state):
    
    global goal_node
    
    explored = set()
    start_state = State(init_state, None, None, 0, 0, 0)
    stack = list([start_state])
    
    print("Start state:")
    print(start_state)
    
    while stack:
        
        node = stack.pop()
        explored.add(node.map)
        
        if node.state == goal_state:
            goal_node = node
            return stack
    
        neighbors = expandChildren(node)
        neighbors.sort(key=lambda s: s.moved, reverse=True)

        for neighbor in neighbors:
            if neighbor.map not in explored:
                stack.append(neighbor)
                explored.add(neighbor.map)
    

In [302]:
# Iterative Deepening Search

# Make a depth-limited search helper function
def dls(explored, stack, max_depth):
    
    node = stack.pop()
    explored.add(node.map)
        
    # If found the goal state
    if node.state == goal_state:
        goal_node = node
        return 'done', explored, stack, max_depth
    
    # If reached maximum depth
    if max_depth <= 0:
        return 'max_depth_reached', explored, stack, max_depth
    
    neighbors = reversed(expand(node))
    for neighbor in neighbors:
        if neighbor.map not in explored:
            stack.append(neighbor)
            explored.add(neighbor.map)
    
    return 'keep_searching', explored, stack, max_depth

    
def ids(init_state):
    
    global goal_node
    
    max_depth = 100
    
    explored = set()
    start_state = State(init_state, None, None, 0, 0)
    stack = list([start_state])
    
    print("Start state:")
    print(start_state)
    
    
    for i in range(max_depth):
        status, exp, sta, md = dls(explored, stack, max_depth)
        dls(exp, sta, md)
    
    
    

In [303]:
def showSolution():
    global goal_node
    global moves
    
    ops = []
    while initial_state != goal_node.state:
        op = (moves[goal_node.move], goal_node)
        ops.insert(0, op)
        goal_node = goal_node.parent

    for o in ops:
        move, state = o
        print(move, "\n", state)
    print("Goal state reached in", len(ops), "moves", "\n\n")

In [304]:
def main():
    
#     print("-------------------------")
#     print("BFS")
#     print("-------------------------")

#     # BFS
#     bfs(initial_state)
#     showSolution()
    
#     print("-------------------------")
#     print("Uniform Cost Search")
#     print("-------------------------")
    
#     # Uniform cost search
#     ucs(initial_state)
#     showSolution()
    
    print("-------------------------")
    print("DFS")
    print("-------------------------")
    
    # DFS
    dfs(initial_state)
    showSolution()
    
#     print("-------------------------")
#     print("Iterative Deepening Search")
#     print("-------------------------")
    
#     # Iterative deepening
#     ids(initial_state)
#     showSolution()
    


if __name__ == '__main__':
    main()


-------------------------
DFS
-------------------------
Start state:
-------------
| 1 | 4 | 2 |
-------------
| 5 | 3 | 0 |
-------------

0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
100
0
0
0
100
100
100
0
0
0
100
100
100
0
100
0
0
0
100
100
100
0
0
0
100
100
100
0
100
0
0
0
100
100
100
0
0
0
100
100
100
0
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
100
0
100
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
100
100
0
0
0
100
0
100
0
Left 
 -------------
| 1 | 4 | 2 |
-------------
| 5 | 0 | 3 |
-------------

Left 
 -------------
| 1 | 4 | 2 |
-------------
| 0 | 5 | 3 |
-------------

Up 
 -------------
| 0 | 4 | 2 |
-------------
| 1 | 5 | 3 |
-------------

Right 
 -------------
| 4 | 0 | 2 |
